In [1]:
import numpy as np
import re

# get the data from bucket 'st-work1-yuchao'
path = 'gs://st-work1-yuchao'
data_from_file = sc.textFile(path + '/author-large.txt', 4)

def extractInformation(row):
    row = row.strip() # remove leading and trailing whitespaces
    words = np.array(row.split("\t")) # split the line into words by tab
    return words

data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(5)

[array(['Jurgen Annevelink', 'Modern Database Systems',
        'Object SQL - A Language for the Design and Implementation of Object Databases.',
        '1995'], dtype='<U78'),
 array(['Rafiul Ahad', 'Modern Database Systems',
        'Object SQL - A Language for the Design and Implementation of Object Databases.',
        '1995'], dtype='<U78'),
 array(['Amelia Carlson', 'Modern Database Systems',
        'Object SQL - A Language for the Design and Implementation of Object Databases.',
        '1995'], dtype='<U78'),
 array(['Daniel H. Fishman', 'Modern Database Systems',
        'Object SQL - A Language for the Design and Implementation of Object Databases.',
        '1995'], dtype='<U78'),
 array(['Michael L. Heytens', 'Modern Database Systems',
        'Object SQL - A Language for the Design and Implementation of Object Databases.',
        '1995'], dtype='<U78')]

In [2]:
data_from_file_conv1 = data_from_file.map(lambda row: np.array(row.strip().split("\t")))
data_from_file_conv2 = data_from_file_conv1.map(lambda row: (row[0], row[2]))

# data_from_file_conv2 has two columns: author and publication title
data_from_file_conv2.take(20)

[('Jurgen Annevelink',
  'Object SQL - A Language for the Design and Implementation of Object Databases.'),
 ('Rafiul Ahad',
  'Object SQL - A Language for the Design and Implementation of Object Databases.'),
 ('Amelia Carlson',
  'Object SQL - A Language for the Design and Implementation of Object Databases.'),
 ('Daniel H. Fishman',
  'Object SQL - A Language for the Design and Implementation of Object Databases.'),
 ('Michael L. Heytens',
  'Object SQL - A Language for the Design and Implementation of Object Databases.'),
 ('William Kent',
  'Object SQL - A Language for the Design and Implementation of Object Databases.'),
 ('Jos A. Blakeley',
  'OQL[C++]: Extending C++ with an Object Query Capability.'),
 ('Yuri Breitbart', 'Transaction Management in Multidatabase Systems.'),
 ('Hector Garcia-Molina', 'Transaction Management in Multidatabase Systems.'),
 ('Abraham Silberschatz', 'Transaction Management in Multidatabase Systems.'),
 ('Stavros Christodoulakis',
  'Multimedia Informa

In [7]:
# the length of the data
data_from_file_conv2.count()

2225370

In [3]:
# collect the data
data = data_from_file_conv2.collect()

In [11]:
# sta1 is a dictionary which keys are publication's title, key value is a list contains the publication's authors
sta1 = {}
# sta2 is a dictionary which structures like key is (author1, author2), key value is the total amount of 
# co-publications author1 and author2 has.
sta2 = {}

def counts(num1, num2):
    for i in range(num1, num2):
        author = data[i][0]
        title = data[i][1]
        if title not in sta1:
            sta1[title] = [author]
        else:
            sta1[title].append(author)
    #print(sta1)

# split the total 2225370 data into pieces which length is 100000
a = []
for i in range(0,2300000,100000):
    a.append(i)
a.append(2225370)
for i in range(len(a)-1):
    counts(a[i],a[i+1])


for alist in sta1.values():
    for k in range(len(alist)):
        for l in range(k+1,len(alist)):
            # make comparision between author1 and author2 which can avoid generate (author2, author1) if
            # (author1, author2) exist in sta2 keys
            if alist[k] > alist[l]: 
                astr = alist[k]+','+alist[l]
                # if (author1, author2) exist, (author1, author2) amounts add 1
                if astr in sta2:
                    sta2[astr] += 1
                # if (author1, author2) don't exist, create (author1, author2) in sta2 and its calue equal to 1
                else:
                    sta2[astr] = 1
            else:
                astr = alist[l]+','+alist[k]
                if astr in sta2:
                    sta2[astr] += 1
                else:
                    sta2[astr] = 1

In [12]:
# make orders with top10 for the dictionary sta2
sta_order = sorted(sta2.items(),key = lambda x:x[1],reverse=True)
print(sta_order[:10])

[('Sudhakar M. Reddy,Irith Pomeranz', 249), ('Divyakant Agrawal,Amr El Abbadi', 161), ('Tomoya Enokido,Makoto Takizawa', 141), ('Henri Prade,Didier Dubois', 122), ('Tharam S. Dillon,Elizabeth Chang', 118), ('Kee-Young Yoo,Hyun-Sung Kim', 111), ('Narayanan Vijaykrishnan,Mary Jane Irwin', 107), ('Mary Jane Irwin,Mahmut T. Kandemir', 100), ('Maurizio Lenzerini,Giuseppe De Giacomo', 99), ('Jiajun Bu,Chun Chen', 99)]
